<h1><center>Trello Connector</center></h1>

### 1. Importing Relevant Modules

In [ ]:
"""Link for Trello code details:
https://developer.atlassian.com/cloud/trello/rest/api-group-boards/#api-boards-id-get"""

#For personal use place in: boardID (for url and url2); keyID; tokenID

#Core Imports
import requests
import json
import pandas as pd
from datetime import date

#Visualising Imports
import matplotlib.pyplot as plt
import numpy as np

#For side by side displays in review (step 6) - not a necessary component
from IPython.display import display, HTML

### 2. Connecting to Trello Board
1. Connects to the relevant URL, tests to ensure appropriate use through key and token and then performs GET function to pull the data

In [ ]:
url = "https://api.trello.com/1/boards/{boardID}/cards"
#After boards/ the string is the same as if you open up the board on Trello. 
#After which you can clarify '/lists' '/cards' etc.

headers = {
  "Accept": "application/json"
}

query = {
  'key': {keyID},
  'token': {tokenID}
}

response = requests.request(
   "GET",
   url,
   headers=headers,
   params=query
)

card_data = json.dumps(json.loads(response.text), sort_keys=True, indent=4, separators=(",", ": "))

each_card_line = card_data.splitlines()

#What the request has pulled:
#for i in each_card_line:
    #print(i)

### Step 3. Collecting Relevant Data
1. Create a function to extract the right info from a line
2. Create lists of info for each of the columns
3. Scroll through the request to take out the relevant lines and then relevant phrase to place into the lists

In [ ]:
#Saves repetition on getting the relevant information out
def clean_line_function(line,number_of_intial_blanks):
    cleaned_lineA = line.strip() #Removes white space
    cleaned_line = cleaned_lineA[number_of_intial_blanks:].strip('",') #Removes fluff, if empty then null description
    return cleaned_line

#Allows counting of lines, useful for selecting relevant lines above or below meeting certain conditions
line_count = 0

#Lists to then place into Pandas
task_id_lst = []
task_name_lst = []
task_status_lst = []
task_type_lst = []
task_description_lst = []
task_assignee_lst = []

for i in each_card_line:    
  
    line_count += 1
    
    #Task ID
    if '"id": ' in i:
        if '"color": ' not in each_card_line[line_count-2]: #Gets rid of label ID being called ID
            task_id_lst.append(clean_line_function(i,9))
    
    #Task Name
    if '"name": ' in i:
        if '"name": ""' != i.strip(): #No tasks are unnamed
            task_name_lst.append(clean_line_function(i,9))   
        
    #Task Status
    if '"idList": ' in i:
        task_status_lst.append(clean_line_function(i,11))
                
    #Task Type
    if '"labels": ' in i: #Not color as this includes 'cover' colors in text, just desire 'labels'
        cleaned_color_line = clean_line_function(each_card_line[line_count+1].strip(),10)
        if "ask " in cleaned_color_line: #Removes no labels, 'ask' means it is moving onto an irrelevant next line
            cleaned_color_line = ""
        task_type_lst.append(cleaned_color_line)
    
    #Task Description
    if '"desc": ' in i:
        task_description_lst.append(clean_line_function(i,9))
         
    #Primary Assignee, note if multiple members to a task - different logic required
    if '"idMembers": ' in i:
        if '"idMembersVoted": ' in each_card_line[line_count]:
            task_assignee_lst.append("Unassigned")
        else:
            task_assignee_lst.append(each_card_line[line_count].strip().strip('"'))

#What the lists collect:
#print(task_id_lst,task_name_lst,task_status_lst,task_type_lst,task_description_lst,task_assignee_lst)

### Step 4. Turn Data into Tables (Pandas DataFrames)
1. Create a dictionary from the lists to convert into Pandas
2. For Task Status Table constructing an artificial primary key and placing today's date as an 2 additional columns

In [ ]:
#Task Table
task_dictionary = {'task_id':task_id_lst,
                   'task_name':task_name_lst,
                   'task_type':task_type_lst,
                   'task_description':task_description_lst}

task_table = pd.DataFrame(task_dictionary)


#Resourcing Table
resourcing_dictionary = {'task_id':task_id_lst,
                         'primary_assignee':task_assignee_lst}

resourcing_table = pd.DataFrame(resourcing_dictionary)


#Task Status Table

     #Constructing PK (as primary key reflects 'task_id' and 'date')
task_status_pk_lst = []
for i in range(1, len(task_id_lst)+1): 
#note if adding sets over time the PK value would be the existing number plus the additional tasks at time of pull
    string_pk_number = 'ts_id' + str(i)
    task_status_pk_lst.append(string_pk_number)


task_status_dictionary = {'task_status_id':task_status_pk_lst,
                         'task_id':task_id_lst,
                         'status':task_status_lst,
                         'date': date.today()}
#Note date was added

task_status_table = pd.DataFrame(task_status_dictionary)

#How the tables look:
#task_table
resourcing_table
#task_status_table

### Step 5. Transforming Data
1) ListID -> List Name ('status')
2) MembershipID ('primary_assignee') -> Person Name
3) Replaces colour of labels with 'Task Type'
4) Review - All 3 tables check

In [ ]:
#1. ListID -> List Name ('status'), Pull list data from trello and compare
url2 = "https://api.trello.com/1/boards/{boardID}/lists"

response2 = requests.request(
   "GET",
   url2,
   headers=headers,
   params=query
)

list_data = json.dumps(json.loads(response2.text), indent=4, separators=(",", ": "))
each_list_line = list_data.splitlines()

#Collecting ID and status data

id_and_status_searching_lst = []
for i in each_list_line:
    if '"id": ' in i:
        id_and_status_searching_lst.append(clean_line_function(i,7))
        
    if '"name": ' in i:
        id_and_status_searching_lst.append(clean_line_function(i,9))
        
def convert_list_to_dictionary(lst):
    res_dct = {lst[i]: lst[i + 1] for i in range(0, len(lst), 2)}
    return res_dct

id_and_status_searching_dictionary = convert_list_to_dictionary(id_and_status_searching_lst)

#Replacing id value with status name (mapping)

task_status_table["status"] = task_status_table["status"].map(id_and_status_searching_dictionary)

In [ ]:
#2. MembershipID ('primary_assignee') -> Person Name
#Finds the unique assigned IDs from the resourcing table and places them in a list to be checked against
member_list = list(dict.fromkeys(resourcing_table['primary_assignee'].tolist()))
member_list.remove("Unassigned")


def find_member_name(member_id):
    url3 = "https://api.trello.com/1/members/"+ member_id
    
    response3 = requests.request(
       "GET",
       url3,
       headers=headers,
       params=query
    )
    
    membership_data = json.dumps(json.loads(response3.text), indent=4, separators=(",", ": "))
    each_member_line = membership_data.splitlines()
    
    #ID doesn't need to be found again
    member_checking_list = [member_id]
    #Counting lines to save effort beyond finding fullName (and preventing a duplication issue)
    count_member_line = 0
    
    for i in each_member_line:
        count_member_line += 1
        
        if '"fullName": ' in i:
            first_clean = clean_line_function(i,7) #Uses earlier function, which doesn't fully do the job
            second_clean = first_clean[6:]
            member_checking_list.append(second_clean)

        if count_member_line == 10:
            break
            
    member_checking_dictionary = convert_list_to_dictionary(member_checking_list) #Runs convert earlier function
    
    #print(member_checking_dictionary)
    
    #Replacing memberID value with full name (mapping)
    resourcing_table["primary_assignee"] = resourcing_table["primary_assignee"].map(member_checking_dictionary)
        
#Cycles the unique members ids through the function which converts their ID to their name
for member_id in member_list:
    find_member_name(member_id)

In [ ]:
#3. Replaces colour of labels with 'Task Type'
task_type_dictionary = {'green':'Data Strategy',
                         'blue':'Data Modelling',
                         'red':'Data Transformation'}

task_table["task_type"] = task_table["task_type"].map(task_type_dictionary)

### Step 6. Data Review
1. Seeing the tables side by side
2. Visualising a couple of use cases:
    1. Number of tasks in a set status - bar chart
    2. Task balance within project - pie graph

In [ ]:
#task_table
#resourcing_table
#task_status_table

print('\033[1m' + '\033[4m' + "Resourcing Table")
display(resourcing_table)
print('\033[1m' + '\033[4m' + "Task Table")
display(task_table)
print('\033[1m' + '\033[4m' + "Task Status Table")
display(task_status_table)

In [ ]:
#1. Number of tasks in a set status - bar chart
status_unique = tuple(task_status_table["status"].unique()) #Unique categories of status

number_of_tasks_lst = [] #Number of tasks in the particular status, counted in the below 3 lines
for i in status_unique:
    valueofresult = task_status_table['status'].value_counts()[i]
    number_of_tasks_lst.append(valueofresult)


    
plt.figure(figsize=(9, 4))
plt.bar(status_unique, number_of_tasks_lst, align='center', alpha=0.5, width=0.7)
plt.xticks(status_unique)
plt.yticks(np.arange(min(number_of_tasks_lst), max(number_of_tasks_lst)+1, 1))
plt.ylabel('Task Count')
plt.title('Status of Tasks')

plt.show()

In [ ]:
#2. Task balance within project - pie graph

pie_graph_labels = tuple(task_table["task_type"].unique())  #Note that as there are no issues / blockers, this doesn't show

number_of_tasks_in_type_lst = [] #Number of tasks in the particular status, counted in the below 3 lines

for i in pie_graph_labels:
    if i == str(i):
        valueofresult2 = task_table["task_type"].value_counts()[i]
        number_of_tasks_in_type_lst.append(valueofresult2)        
    else: #Null values must be counted differently, their tell here is they're not string values
        count_nan = len(task_table["task_type"]) - task_table["task_type"].count()
        number_of_tasks_in_type_lst.append(count_nan)

proportions_lst = []
for i in number_of_tasks_in_type_lst:
    proportions = i * 100/len(task_table)
    proportions_lst.append(proportions)
    
sizes = proportions_lst

explode_lst = [0]*len(pie_graph_labels)
explode_lst[1] = 0.1
pie_explode = explode_lst

fig1, ax1 = plt.subplots()
ax1.pie(sizes, explode=pie_explode, labels=pie_graph_labels, autopct='%1.1f%%',
        shadow=True, startangle=90)
ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

plt.show()